# 0819

## 서설

프롬프트 > api > llm 모델

api - chat.completion etc

기본적인 것은 되지만, 헐루시네이션, 학습되지 않은 부분이 문제

에이젼트를 만들려면 위의 문제를 처리해야 함 - 정확한 정보를 줘야 함

이번주는 rag(retreival augmented generation) 모델을 학습시키는 방법을 배울 것

rag는 학습과 상관없이 정보만 가져오는 모델

고정된 정보를 학습시키는 것은 fine-tuning

래그와 비슷한 것이 function call - 함수를 호출하는 것 , 사용자 함수를 사용할 수 있음, 예) 인기 앨범차트에 대한 정보를 래그로 가져올 수있지만 펑션콜로 찾을 수도 있음, 디비에 이미 저장된 정보를 가져올 수 있는 함수를 만들어서 사용 , 실시간 날씨 정보를 가져오는 함수를 만들어서 사용- 위치를 알려주면 날씨 정보를 가져오는 함수를 만들어서 사용(날씨 사이트의 api를 사용해서)

rag, fine-tuning, function call를 사용하면 기존 모델을 확장할 수 있음

랭체인 , 랭스미스 배울 것

## function call

https://platform.openai.com/docs/guides/function-calling

In [1]:
from openai import OpenAI
import os
MODEL="gpt-4o-mini-2024-07-18"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

### Function Calling의 사용
- 이 코드에서 function calling은 모델이 적절한 시점에 `외부 함수`(get_current_weather)를 호출하도록 유도합니다. 모델이 What is the current weather in {user_location}?와 같은 질문에 응답할 때, 해당 함수가 정의되어 있으면 모델은 그 함수를 호출하여 날씨 정보를 가져오려 합니다.

- function_call="auto" 설정을 통해 GPT 모델은 적절한 함수가 있을 때 자동으로 그 함수를 호출할 수 있습니다. 여기서는 get_current_weather 함수 명세를 통해 사용자가 입력한 위치에 맞는 날씨 정보를 가져올 수 있도록 유도됩니다.

- 만약 모델이 함수 호출을 결정하면, 응답에 function_call이 포함되며, 이때 함수 이름과 그 인자(user_location)가 함께 반환됩니다. 그런 후에 함수가 호출되고, 결과가 다시 모델에 제공됩니다.

이 과정에서 모델은 단순히 인자를 생성하고, 함수를 호출하는 것은 개발자의 책임입니다. OpenAI의 API는 실제 함수 실행을 수행하지 않으므로, 함수 호출 후 결과를 처리하고 다시 모델에 넘기는 과정을 수동으로 처리하게 됩니다.

In [2]:
import openai
import json
import requests
from openai import OpenAI

weather_api_key = os.environ.get("OPENWEATHER_API_KEY")

def get_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    return response.json()

messages = [
    {'role': 'system', 'content': 'you are a helpful assistant'},
    {'role': 'user', 'content': 'what is the weather in New York?'},
]

response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    functions=[{
        "name": "get_weather",
        "description": "Get the weather for a location",
        "parameters": {
            "type": "object",
            "properties":{
                "location": {
                    "type": "string",
                    "description": "The location to get the weather for"
                }
            },
            "required": ["location"]
        }
    }],
    function_call="auto"
)
# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다. - 도구 호출 포함
response_message = response.choices[0].message
messages.append(response_message)  # 답변을 메시지에 추가, 다음 대화에 사용
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"New York"}', name='get_weather'), tool_calls=None)


In [3]:
# 도구 호출 여부 확인
function_call = response_message.function_call
if function_call:
    tool_function_name = function_call.name
    tool_arguments = json.loads(function_call.arguments)
    
    if tool_function_name == "get_weather":
        location = tool_arguments["location"]
        weather_results = get_weather(location)
        print(weather_results)
        messages.append({
            "role": "function",
            "name": tool_function_name,
            "content": json.dumps(weather_results)
        })
        
        model_response_with_function_call = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        print(model_response_with_function_call.choices[0].message.content)
        
    else:
        print("Unknown function call")
        
else:
    print(response_message.content)

{'coord': {'lon': -74.006, 'lat': 40.7143}, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10n'}, {'id': 701, 'main': 'Mist', 'description': 'mist', 'icon': '50n'}], 'base': 'stations', 'main': {'temp': 21.62, 'feels_like': 22.19, 'temp_min': 20.33, 'temp_max': 23.64, 'pressure': 1008, 'humidity': 90, 'sea_level': 1008, 'grnd_level': 1007}, 'visibility': 4828, 'wind': {'speed': 4.63, 'deg': 50, 'gust': 7.72}, 'rain': {'1h': 3.87}, 'clouds': {'all': 100}, 'dt': 1724029346, 'sys': {'type': 1, 'id': 4610, 'country': 'US', 'sunrise': 1723975794, 'sunset': 1724024997}, 'timezone': -14400, 'id': 5128581, 'name': 'New York', 'cod': 200}
The current weather in New York is as follows:

- **Condition:** Moderate rain and mist
- **Temperature:** 21.6°C (feels like 22.2°C)
- **Humidity:** 90%
- **Wind Speed:** 4.6 m/s from the northeast
- **Visibility:** Approximately 4828 meters
- **Rainfall in the last hour:** 3.87 mm
- **Cloud Cover:** 100%

It seems to be a ra

In [10]:
import openai
import json
import requests
from openai import OpenAI

weather_api_key = os.environ.get("OPENWEATHER_API_KEY")

location_input = input("Enter a location: ")

def get_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    return response.json()

messages = [
    {"role": "system", "content": "you are a helpful assistant"},
    {"role": "user", "content": f"what is the weather in {location_input}?"},
]

response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    functions=[
        {
            "name": "get_weather",
            "description": "Get the weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location to get the weather for",
                    }
                },
                "required": ["location"],
            },
        }
    ],
    function_call="auto",
)
# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다. - 도구 호출 포함
response_message = response.choices[0].message
messages.append(response_message)  # 답변을 메시지에 추가, 다음 대화에 사용

# 도구 호출 여부 확인
function_call = response_message.function_call
if function_call:
    tool_function_name = function_call.name
    tool_arguments = json.loads(function_call.arguments)

    if tool_function_name == "get_weather":
        location = tool_arguments["location"]
        weather_results = get_weather(location)
        messages.append(
            {
                "role": "function",
                "name": tool_function_name,
                "content": json.dumps(weather_results),
            }
        )
        messages.append(
            {
                "role": "user",
                "content": "위 내용 한국어로 해줘",
            }
        )

        model_response_with_function_call = client.chat.completions.create(
            model=MODEL, messages=messages
        )
        print(model_response_with_function_call.choices[0].message.content)

    else:
        print("Unknown function call")

else:
    print(response_message.content)

서울의 현재 날씨는 다음과 같습니다:

- **날씨:** 구름 약간
- **기온:** 31.74°C 
- **체감 온도:** 38.74°C 
- **최저 기온:** 30.69°C 
- **최고 기온:** 31.76°C 
- **습도:** 70%
- **기압:** 1009 hPa
- **바람 속도:** 1.54 m/s 
- **바람 방향:** 170도
- **가시 거리:** 10 km
- **구름 양:** 20%

오늘 서울의 날씨는 더운 느낌이며, 몇 가지 구름이 있습니다.


In [11]:
import openai
import json
import requests
from openai import OpenAI

weather_api_key = os.environ.get("OPENWEATHER_API_KEY")

location_input = input("Enter a location: ")


def get_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    return response.json()


messages = [
    {"role": "system", "content": "you are a helpful assistant"},
    {"role": "user", "content": f"what is the weather in {location_input}?"},
]

response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    functions=[
        {
            "name": "get_weather",
            "description": "Get the weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location to get the weather for", #정확하게 기술해줘야 location을 리턴받을 수 있음
                    }
                },
                "required": ["location"],
            },
        }
    ],
    function_call="auto",
)
# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다. - 도구 호출 포함
response_message = response.choices[0].message
messages.append(response_message)  # 답변을 메시지에 추가, 다음 대화에 사용

# 도구 호출 여부 확인
function_call = response_message.function_call
if function_call:
    tool_function_name = function_call.name
    tool_arguments = json.loads(function_call.arguments)

    if tool_function_name == "get_weather":
        location = tool_arguments["location"]
        weather_results = get_weather(location)
        messages.append(
            {
                "role": "system",
                "content": json.dumps(weather_results),
            }
        )
        messages.append(
            {
                "role": "user",
                "content": "위 내용 한국어로 해줘",
            }
        )

        model_response_with_function_call = client.chat.completions.create(
            model=MODEL, messages=messages
        )
        print(model_response_with_function_call.choices[0].message.content)

    else:
        print("Unknown function call")

else:
    print(response_message.content)

현재 서울의 날씨는 다음과 같습니다:

- 기온: 31.74°C (체감 온도: 38.74°C)
- 최저 기온: 30.69°C
- 최고 기온: 31.76°C
- 습도: 70%
- 기압: 1008 hPa
- 시정 거리: 10,000 m
- 바람 속도: 1.54 m/s (방향: 남쪽)
- 구름 상태: 약간의 구름 (20% 구름)

날씨는 대체로 맑은 편입니다.


### How to call functions with model generated arguments

다음 예제에서는 모델에서 생성된 입력을 갖는 함수를 실행하는 방법을 보여주고 이를 사용하여 데이터베이스에 대한 질문에 답할 수 있는 에이전트를 구현합니다. 단순화를 위해 Chinook 샘플 데이터베이스를 사용하겠습니다 .

참고: 모델이 올바른 SQL을 생성하는 데 완벽하게 신뢰할 수 없기 때문에 프로덕션 환경에서 SQL 생성은 위험할 수 있습니다.

이 코드는 OpenAI의 GPT 모델을 사용하여 SQLite 데이터베이스에서 음악 관련 질문에 대한 답변을 SQL 쿼리로 변환한 후, 해당 쿼리를 실행하여 결과를 반환하는 방식으로 function calling을 구현한 예시입니다. 여기서는 사용자가 앨범 관련 질문을 하면, GPT-4가 질문을 SQL 쿼리로 변환하고, SQLite 데이터베이스를 조회하여 결과를 반환합니다.

쿼리 실행을 위해 sqlite3 라이브러리를 사용합니다. 이 라이브러리는 Python 표준 라이브러리에 포함되어 있으므로 별도 설치가 필요하지 않습니다.

chinook.db 에 연결하여 데이터베이스를 조회하고, 사용자가 질문한 내용에 대한 답변을 반환하는 함수를 정의합니다.

In [13]:
import sqlite3

conn = sqlite3.connect(r"D:\pythonProject\ML\data\Chinook.db")

데이터베이스 테이블 및  열 정보 조회
- 데이터베이스에는 다음과 같은 테이블이 있습니다.
    - albums: 앨범 정보
    - artists: 아티스트 정보
    - tracks: 트랙 정보
    - genres: 장르 정보
    - media_types: 미디어 타입 정보
    - playlists: 플레이리스트 정보
    - playlist_track: 플레이리스트와 트랙 정보
- get_table_columns 함수는 데이터베이스 테이블의 열 정보를 조회합니다
- 이 정보는 나중에 GPT 모델이 SQL 쿼리를 생성할 때 사용할 스키마 정보를 제공하는 데 활용됩니다.

In [15]:
def get_table_names(conn):
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names

def get_column_names(conn, table_name):
    column_names = []
    columns = conn.execute("PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names

def get_database_info(conn):
    table_dicts =[]
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

데이터베이스의 테이블과 열 정보를 문자열 형태로 저장하여, 나중에 sql쿼리 작성시 참조할 수 있도록 합니다.

In [19]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"{table['table_name']}\nColumns: {', '.join(table['column_names'])}" 
        for table in database_schema_dict
    ]
)
print(database_schema_string)

Album
Columns: 
Artist
Columns: 
Customer
Columns: 
Employee
Columns: 
Genre
Columns: 
Invoice
Columns: 
InvoiceLine
Columns: 
MediaType
Columns: 
Playlist
Columns: 
PlaylistTrack
Columns: 
Track
Columns: 


tools라는 리스트에 ask_database라는 함수 명세를 정의합니다.
- 이 함수는 사용자의 질문에 맞는 SQL 쿼리를 받아 데이터베이스에서 정보를 조회하고, 이를 반환하는 기능을 합니다.
- 함수의 매개변수로 query가 있으며, 이는 SQL 쿼리를 텍스트 형태로 전달받아 실행하는 구조입니다.
- 중요한 점은 database_schema_string이 함수 설명에 포함되어 있어, 모델이 데이터베이스 스키마에 맞는 SQL 쿼리를 생성할 수 있도록 도움을 줍니다.

In [22]:
tools = [
    {
    "type": "function",
    "function": {
        "name":"ask_database",
        "description": "use this function to answer questions about music database, input should be a fully formed SQL query",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                        SQL query extracting info to answer the user's question.
                        SQL should be written using this database schema:
                        {database_schema_string}
                        The query should be returned in sql format, not as a JSON object.
                    """
                }
            },
            "required": ["query"],
        },
        }
    }
]

Executing SQL queries

이제 실제로 데이터베이스에 대한 쿼리를 실행하는 함수를 구현해 보겠습니다.

In [23]:
def ask_database(conn,query):
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f'query failed: {e}'
    return results

##### Steps to invoke a function call using Chat Completions API:

- 1단계 : 모델이 사용할 도구를 선택하도록 하는 내용으로 모델을 프롬프트합니다. 함수 이름 및 서명과 같은 도구에 대한 설명은 '도구' 목록에 정의되어 API 호출에서 모델에 전달됩니다. 선택한 경우 함수 이름과 매개변수가 응답에 포함됩니다.
- 2단계 : 모델이 함수를 호출하려고 하는지 프로그래밍적으로 확인합니다. 참이면 3단계로 진행합니다.
- 3단계 : 응답에서 함수 이름과 매개변수를 추출하고 매개변수와 함께 함수를 호출합니다. 결과를 메시지에 추가합니다.
- 4단계 : 메시지 목록으로 채팅 완료 API를 호출하여 응답을 가져옵니다.

In [29]:
messages = [{
    "role": "user",
    "content": "what is the name of the album with the most tracks?"
}]

response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools,
    tool_choice="auto" #모델이 자동으로 함수 호출을 결정, sql 쿼리로 변환할 필요가 있다고 판단하면 자동으로 함수 호출
)

response_message = response.choices[0].message
messages.append(response_message)
print(response_message)
import pprint
pprint.pprint(response_message.tool_calls)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_r4rnUUR21RmyuTW5vjX5Lawj', function=Function(arguments='{"query":"SELECT Album.Title FROM Album JOIN (SELECT AlbumId, COUNT(*) AS TrackCount FROM Track GROUP BY AlbumId ORDER BY TrackCount DESC LIMIT 1) AS TrackCounts ON Album.AlbumId = TrackCounts.AlbumId;"}', name='ask_database'), type='function')])
[ChatCompletionMessageToolCall(id='call_r4rnUUR21RmyuTW5vjX5Lawj', function=Function(arguments='{"query":"SELECT Album.Title FROM Album JOIN (SELECT AlbumId, COUNT(*) AS TrackCount FROM Track GROUP BY AlbumId ORDER BY TrackCount DESC LIMIT 1) AS TrackCounts ON Album.AlbumId = TrackCounts.AlbumId;"}', name='ask_database'), type='function')]


## ChatCompletionMessageToolCall

- **id**: `'call_r4rnUUR21RmyuTW5vjX5Lawj'`
- **type**: `'function'`
- **function**:
  - **name**: `'ask_database'`
  - **arguments**:
    ```json
    {
      "query": 
      "SELECT Album.Title FROM Album JOIN (SELECT AlbumId, COUNT(*) AS TrackCount FROM Track GROUP BY AlbumId ORDER BY TrackCount DESC LIMIT 1) AS TrackCounts ON Album.AlbumId = TrackCounts.AlbumId;"
    }
    ```

```sql
SELECT Album.Title FROM Album JOIN (SELECT AlbumId, COUNT(*) AS TrackCount FROM Track GROUP BY AlbumId ORDER BY TrackCount DESC LIMIT 1) AS TrackCounts ON Album.AlbumId = TrackCounts.AlbumId;
```

In [27]:
tool_calls = response_message.tool_calls

if tool_calls:
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = json.loads(tool_calls[0].function.arguments)["query"]
    
    # 도구 호출이 ask_database인 경우 쿼리를 실행하고 결과를 리스트에 추가
    if tool_function_name == "ask_database":
        results = ask_database(conn, tool_query_string)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call_id,
            "name": tool_function_name,
            "content": results
        })
        
        # 도구 호출을 포함한 메시지로 모델에 대화를 요청
        model_response_with_function_call = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f"Error: {tool_function_name} Unknown function call")
else:
    print(response_message.content)

The album with the most tracks is titled "Greatest Hits."


rag는 검색할 데이터를 벡터화해서 가져옴

비정형 데이터는 래그로 가져옴

정형 데이터는 펑션콜로 가져오는게 좋음